In [22]:
import openai
import pandas as pd
from openai import OpenAI
import numpy as np

In [23]:
df=pd.read_csv(r"E:\bigalien\data.csv")

In [24]:
df

,text,category
0,WASHINGTON (Reuters) head conservative Republi...,1
1,WASHINGTON (Reuters) Transgender people allowe...,1
2,WASHINGTON (Reuters) special counsel investiga...,1
3,WASHINGTON (Reuters) Trump campaign adviser Ge...,1
4,SEATTLE/WASHINGTON (Reuters) President Donald ...,1
...,...,...
44884,21st Century Wire says 21WIRE reported earlier...,0
44885,21st Century Wire says familiar theme. Wheneve...,0
44886,Patrick Henningsen 21st Century WireRemember O...,0
44887,21st Century Wire says Al Jazeera America go h...,0


In [25]:
import os
os.environ["OPENAI_API_KEY"] = "这里填入自己的openai api"
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [26]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import numpy as np

# 创建OpenAI客户端
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# 初始化一个空列表来存储嵌入向量
embeddings = []

# 使用tqdm创建进度条
for i in tqdm(range(len(df)), desc="Processing"):
    # 处理单行数据
    embedding = get_embedding(df.iloc[i]['text'], model='text-embedding-ada-002')
    embeddings.append(embedding)
    
    # 每1000次迭代保存一次
    if (i + 1) % 1000 == 0 or i == len(df) - 1:  # 最后一次迭代也保存
        # 将最后1000个嵌入向量保存为NumPy文件
        np.save(f'output/embeddings_batch_{i // 1000}.npy', np.array(embeddings[-1000:]))
        if i != len(df) - 1:  # 如果不是最后一次迭代，清空嵌入向量列表
            embeddings = []

Processing: 100%|██████████████████████████████████████████████████████████████| 44889/44889 [5:30:29<00:00,  2.26it/s]


In [32]:
import numpy as np

# 初始化一个空列表来存储加载的数据
all_embeddings = []

# 循环从0到44加载每个文件
for i in range(45):
    batch_embeddings = np.load(f'output/embeddings_batch_{i}.npy')
    all_embeddings.append(batch_embeddings)

# 使用vstack将所有嵌入向量垂直堆叠起来
all_embeddings_stacked = np.vstack(all_embeddings)

In [33]:
all_embeddings_stacked.shape

(44889, 1536)

In [34]:
np.save('gpt3emb.npy', all_embeddings_stacked)